In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [46]:
import pandas as pd
import glob
import bz2
from pprint import pprint
import json
import os
import csv
import multiprocessing as mp
import time

scp stat1003.eqiad.wmnet:/srv/ellery/talk_diffs/enwiki-20160113-pages-meta-history10.xml-*.bz2 ./wikipedia/data/

In [ ]:
rsync -av /srv/public-datasets/enwiki/user_talk_diffs stat1002.wikimedia.org::a/home/ellery
        
rsync -av stat1003.wikimedia.org::srv/public-datasets/enwiki/user_talk_diffs .


In [3]:
path_glob = '../data/enwiki-20160113-pages-meta-history10.xml-*.bz2'

In [4]:
output_dir = '../data'

In [5]:
files = glob.glob(path_glob)

In [30]:
def get_features(obj):
    f = {}
    if 'comment' in obj:
        f['rev_comment'] = obj['comment']
        
    f['insert_only'] = 1
    if 'diff' in obj:
        if 'ops' in obj['diff']:
            for e in obj['diff']['ops']:
                if e['name'] == 'insert':
                    if 'tokens' in e:
                        insertion = ''.join(e['tokens'])
                        if len(insertion) > 0:
                            f['insertion'] = insertion
                else:
                    f['insert_only'] = 0
    if 'id' in obj:
        f['rev_id'] = obj['id']
        
    if 'page' in obj:
        if 'id' in obj['page']:
            f['page_id'] = obj['page']['id']
        if 'title' in obj['page']:
            f['page_title'] = obj['page']['title']
            
    if 'timestamp' in obj:
        f['rev_timestamp'] = obj['timestamp']
        
        
    if 'user' in obj:
        if 'id' in obj['user']:
            f['user_id'] = obj['user']['id']
        if 'text' in obj['user']:
            f['user_text'] = obj['user']['text']
    return f
        
        
                        

In [44]:
def convert(infile):
    outfile = os.path.join(output_dir, infile.split('/')[-1].replace('bz2', 'tsv.bz2'))
    with bz2.open(infile, 'rt') as fin, bz2.open(outfile, 'wt') as fout:
        fieldnames = ['rev_comment', 'insertion', 'insert_only', 'rev_id', 'page_id', 'page_title', 'rev_timestamp', 'user_id', 'user_text']
        writer = csv.DictWriter(fout, fieldnames=fieldnames, delimiter='\t')
        writer.writeheader()
        for line in fin:
            obj = json.loads(line)
            f = get_features(obj)
            writer.writerow(f)
    

In [47]:
p = mp.Pool(mp.cpu_count())
p.map(convert, files)

[None, None, None, None, None, None, None, None]

In [41]:
df = pd.read_csv(bz2.open(outfile, 'rt'), sep = '\t')

In [42]:
df.shape

(59825, 9)

In [43]:
df[df['insert_only'] == 1].head(100)

,rev_comment,insertion,insert_only,rev_id,page_id,page_title,rev_timestamp,user_id,user_text
0,User talk:Ambi moved to User talk:Ambi/Archive3,#REDIRECT [[User talk:Ambi/Archive3]]\n,1,5404708,925005,Ambi~enwiki/Archive4,2004-08-24T04:07:52Z,13040,Vanished user 5zariu3jisj0j4irj
120,your tests,{{test}} P.S.: warning: if you keep making tes...,1,5404915,925015,66.57.43.214,2004-08-24T04:18:26Z,4444,Infrogmation
123,blank ancient IP talk page posts per [[Wikiped...,NaN,1,259804119,925015,66.57.43.214,2008-12-23T22:57:29Z,196446,BD2412
124,"Welcome to Wikipedia, [[User: 81.86.251.101]] !","Dear [[User: 81.86.251.101]],\n\nWelcome to Wi...",1,16716724,925016,81.86.251.101,2004-08-24T04:18:31Z,63672,PFHLai
125,"blank ancient IP talk page warnings, Replaced:...",NaN,1,257862579,925016,81.86.251.101,2008-12-14T07:17:27Z,196446,BD2412
126,"Hey there, anarchist!","Welcome to Wikipedia, Carlton. You will proba...",1,5456045,925045,Carltonh,2004-08-24T04:34:01Z,45825,Jdavidb
150,NaN,== Note to self ==\n\nI do not understand the ...,1,5405319,925078,Ilyanep/Talk Archive 3,2004-08-24T04:53:51Z,10837,Ilyanep
156,Navbar thing,"[[User talk:Ilyanep|Ilyanep's Talk]] , [[User ...",1,6054324,925086,Ilyanep/TA,2004-08-24T04:58:22Z,10837,Ilyanep
168,your test,{{test}} -- [[User:Infrogmation|Infrogmation]]...,1,5979548,925096,64.12.117.18,2004-08-24T05:08:13Z,4444,Infrogmation
169,NaN,NaN,1,6020861,925096,64.12.117.18,2004-09-19T02:22:21Z,NaN,64.12.116.10


In [50]:
for i, r in df[df['insert_only'] == 1][:100].iterrows():
    print(r['insertion'])
    print('\n\n\n\n')

#REDIRECT [[User talk:Ambi/Archive3]]






{{test}} P.S.: warning: if you keep making tests in the middle of articles, it can be considered vandalism and may result in your being blocked from editing. -- [[User:Infrogmation|Infrogmation]] 04:18, 24 Aug 2004 (UTC)





nan





Dear [[User: 81.86.251.101]],

Welcome to Wikipedia !  Your test at [[Gastrointestinal tract]] have worked, eh ?  Please stop, and instead, continue with your testing at [[Wikipedia:Sandbox]].

Please consider [[Wikipedia:Why create an account?|setting up your own account]].  It's more fun this way.  :-)

Happy editing !<br>
-- [[User:PFHLai|PFHLai]] 04:18, 2004 Aug 24 (UTC)





nan





Welcome to Wikipedia, Carlton.  You will probably never get out. :) [[User:Jdavidb|Jdavidb]] 04:34, 24 Aug 2004 (UTC)





== Note to self ==

I do not understand the meaning or the fundamental background of your words, or If I do, I stand in shock and disbelief [[User:Ilyanep|Ilyanep]] [[User talk:Ilyanep|(Talk)]] 17:13, 23 Ju